In [ ]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
import statsmodels
import numpy as np


#
data = pd.read_csv('Datasets/classified_data1.csv',error_bad_lines=False, index_col=False, dtype='unicode', nrows=195081)
#data = data.query("state='Alabama'")

data.sort_values("likeCount", inplace = True)
data["likeCount"] = pd.to_numeric(data["likeCount"])
data["replyCount"] = pd.to_numeric(data["replyCount"])
data["retweetCount"] = pd.to_numeric(data["retweetCount"])
app = JupyterDash(__name__)
data["date"] = data["date"].astype('datetime64[ns]')
data["Engagement"] = data["likeCount"] + data["replyCount"] + data["retweetCount"]


colour_sequence = px.colors.qualitative.Vivid
#testing queries and combinations
available_indicators = ["date","likeCount","replyCount","retweetCount","start date","sentiment","Engagement"]
temp_df = pd.DataFrame(data = data["likeCount"].value_counts(bins=1000))
#df2[df2['rr_quality'] < 100].groupby([df2.index.hour,'sleep_summary_id'])
sample_query = pd.DataFrame(data.groupby(["likeCount"]).size().reset_index().rename(columns={0:'Tweet Count'}))
#print(data.freq.value_counts(bins=20).sort_index())
pie_graph_data =  sample_query[sample_query['Tweet Count'] > (sample_query['Tweet Count'].sum()/100)]
pie_graph_data_extra = sample_query[sample_query['Tweet Count'] < (sample_query['Tweet Count'].sum()/100)]
pie_graph_data = pie_graph_data.append(pd.DataFrame([[str(int(pie_graph_data_extra.iloc[0]["likeCount"]))+"+", pie_graph_data_extra['Tweet Count'].sum()]], columns=["likeCount","Tweet Count"]))
print(pie_graph_data)
#print(sample_query[sample_query['Tweet Count'] < (sample_query['Tweet Count'].sum()/100)])
#print(pd.cut(sample_query["Tweet Count"],6,duplicates='raise').value_counts())



In [ ]:


app.layout = html.Div([
    
    dcc.Tabs([
        dcc.Tab(label='Scatter Plot', children=[
            
            #Scatter plot
            html.Div([
                html.Div(style={'height': 20}),
                html.Div([
                    dcc.Dropdown(
                        id="xaxis-column",
                        options = [{'label': i, 'value': i} for i in available_indicators],
                        value="date"
                    ),
                    #Radio item functionality not implemented yet
                    dcc.RadioItems(
                        id="xaxis-type",
                        options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                        value='Linear',
                        labelStyle={'display': 'inline-block'}

                    )


                ]),
                html.Div(style={'height': 20}),
                html.Div([
                    dcc.Dropdown(
                        id="yaxis-column",
                        options = [{'label': i, 'value': i} for i in available_indicators],
                        value="likeCount"
                    ),
                    #Radio item functionality not implemented yet
                    dcc.RadioItems(
                        id="yaxis-type",
                        options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                        value='Linear',
                        labelStyle={'display': 'inline-block'}

                    )


                ])
            ]),



            dcc.Graph(id="over-time-scatter"),
            html.Div([
                "Min - x:  ",
                dcc.Input(id="minimum-x-input", value=0, type='number',min=0),
                "    ",
                "Max - x: ",
                dcc.Input(id="maximum-x-input",value=100,type='number',min=0)]),
            html.Div(style={'height': 10,'display': 'inline-block'}),
            html.Div([
                "Min - y:  ",
                dcc.Input(id="minimum-y-input", value=0, type='number', min=0),
                " ",
                "Max - y: ",
                dcc.Input(id="maximum-y-input", value=100,type='number', min=0)]),
            dcc.Dropdown(
                        id="colour-column",
                        options = [{'label': i, 'value': i} for i in available_indicators + ["state"]],
                        value="sentiment"
                    ),
             html.Div([
                "Query: ",
                dcc.Input(id="scatter-plot-query", value='state==state', type='text'),
            ]),
            
        ]),
        
        dcc.Tab(label='Bar Graph', children=[
            html.H3('Tab content 2'),
            dcc.Graph(id="Pie chart",
                     figure=px.pie(pie_graph_data,values="Tweet Count", names="likeCount",title="Tweet quantity by like count")
                      
                     ),
        ]),
        
        dcc.Tab(label='Line Graph', children=[
            #bar-graph?
            html.H3('Line graph'),
            dcc.Graph(id="line-graph"),
            html.Div([
                "Query: ",
                dcc.Input(id="line-graph-query", value='state=="Alabama"', type='text'),
                " Variable: ",
                dcc.Input(id="line-graph-x-value", value='start date', type='text'),
                " Additional function to apply ",
                dcc.Input(id="line-graph-function", value='', type='text'),
            ]),
        ]),
    ]),
    
    ]
    
)

@app.callback(
    Output("over-time-scatter", "figure"),
    Input('xaxis-column', 'value'),
    Input('yaxis-column', 'value'),
    Input('xaxis-type', 'value'),
    Input('yaxis-type', 'value'),
    #[Input("x-range-slider", "value")],
    Input("minimum-x-input", "value"),
    Input("maximum-x-input", "value"),
    Input("minimum-y-input", "value"),
    Input("maximum-y-input", "value"),
    Input("colour-column", "value"),
    Input("scatter-plot-query","value"),

)
def update_scater_plot(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                min_x, max_x,
                min_y,max_y,
                colour, query):
    try:
        data_query = data.query(query)
    except:
        data_query = data
    if xaxis_column_name == 'date':
        mask_x = (data_query["date"] >= data_query["date"].min().replace(hour=0, minute=0, second=0) + pd.Timedelta(days=min_x)) & (data_query["date"] <= data_query["date"].min().replace(hour=0, minute=0, second=0) + pd.Timedelta(days=max_x))
    else:
         mask_x = (data_query[xaxis_column_name] >= min_x) & (data_query[xaxis_column_name] <= max_x)
    if yaxis_column_name == 'date':
        mask_y = (data_query["date"] >= data_query["date"].min().replace(hour=0, minute=0, second=0) + pd.Timedelta(days=min_y)) & (data_query["date"] <= data_query["date"].min().replace(hour=0, minute=0, second=0) + pd.Timedelta(days=max_y))
    else:
        mask_y = (data_query[yaxis_column_name] >= min_y) & (data_query[yaxis_column_name] <= max_y)
    #mask is broken when using same data on both axis - not really a problem
    mask = mask_x & mask_y
    fig = px.scatter(
        data_query[mask], x=xaxis_column_name, y=yaxis_column_name, color=colour,
            color_discrete_sequence=colour_sequence, trendline="ols")
    return fig

@app.callback(
    Output("line-graph", "figure"),
    Input("line-graph-query","value"),
    Input("line-graph-x-value", 'value'),
    Input("line-graph-function", 'value')
)
def update_line_graph(query,xs,function):
    try:
        sample_query = pd.DataFrame(data.query(query).groupby(xs.split(",")).size().reset_index().rename(columns={0:'Tweet Count'}))
        try:
            sample_query = sample_query.apply(function).fillna(0)
        except:
            None
        if(len(xs.split(","))>1):
            fig = px.line(sample_query, y='Tweet Count', x=xs.split(",")[0], color = xs.split(",")[1],title="Test line graph",
            color_discrete_sequence=colour_sequence)
        else:
            fig = px.line(sample_query, y='Tweet Count', x=xs.split(",")[0],title="Test line graph",
            color_discrete_sequence=colour_sequence)
        return fig
    except:
        raise PreventUpdate


In [4]:
app.run_server(mode='external', port=1002)

Dash app running on http://127.0.0.1:1002/


# 